In [5]:
# Import modules
import pandas as pd
import json
import requests
import pprint
import os

# Career One Stop API credentials
from config import token
from config import userId

# College Scorecard API key
from config import api_key

# Gather & Clean Data
## Step 1: Get list of occupations
Source: https://www.onetonline.org/find/career?c=11&g=Go

In [6]:
# Pull occupations from csv file
file_path = os.path.join('Resources','Information_Technology.csv')
o_net_df =  pd.read_csv(file_path, encoding="ISO-8859-1")

## Step 2: Get Occupation Wages & Education
Source: https://www.careeronestop.org/Developers/WebAPI/web-api.aspx

In [7]:
# Configure Header
headers = {'Authorization': f'Bearer {token}'}

# Global API Param(s)
location = 'FL'

occupation_df = pd.DataFrame()
i = 0
for code in o_net_df['Code']:

    # API call to retrieve occupation details
    keyword = o_net_df['Code'][i]
    params = {'wages': 'true'
             ,'training': 'true'
             }
    api_url = f'https://api.careeronestop.org/v1/occupation/{userId}/{keyword}/{location}?'
    response = requests.get(api_url, headers=headers, params=params).json()
 
    # Loop through wage rates & assign the annual & hourly valiables accordingly
    annual = 0
    hourly = 0
    for row in response['OccupationDetail'][0]['Wages']['StateWagesList']:
        if row['RateType'] == 'Annual':
            annual = row
        elif row['RateType'] == 'Hourly':
            hourly = row
    
    # Loop through education levels & assign valiables accordingly
    EducationLevel_1 = 0
    EducationLevel_2 = 0
    EducationLevel_3 = 0
    EducationLevel_4 = 0
    EducationLevel_5 = 0
    EducationLevel_6 = 0
    EducationLevel_7 = 0

    for row in response['OccupationDetail'][0]['EducationTraining']['EducationType']:
        if row['EducationLevel'] == 'Less than high school diploma':
            EducationLevel_1 = row['Value']
        elif row['EducationLevel'] == 'High school diploma or equivalent':
            EducationLevel_2 = row['Value']
        elif row['EducationLevel'] == 'Some college, no degree':
            EducationLevel_3 = row['Value']
        elif row['EducationLevel'] == "Associate's degree":
            EducationLevel_4 = row['Value']
        elif row['EducationLevel'] == "Bachelor's degree":
            EducationLevel_5 = row['Value']
        elif row['EducationLevel'] == "Master's degree":
            EducationLevel_6 = row['Value']
        elif row['EducationLevel'] == 'Doctoral or professional degree':
            EducationLevel_7 = row['Value']
    
    # Build DataFrame with this instance
    df = pd.DataFrame(
            {'Occupation':[o_net_df['Occupation'][i]]
            ,'Career Pathway':o_net_df['Career Pathway'][i]
            ,'OnetTitle':[response['OccupationDetail'][0]['OnetTitle']]
            ,'OnetDescription':[response['OccupationDetail'][0]['OnetDescription']]
            ,'Annual Wage - Pct10':[annual['Pct10']]
            ,'Annual Wage - Pct25':[annual['Pct25']]
            ,'Annual Wage - Median':[annual['Median']]
            ,'Annual Wage - Pct75':[annual['Pct75']]
            ,'Annual Wage - Pct90':[annual['Pct90']]
            ,'Hourly Wage - Pct10':[hourly['Pct10']]
            ,'Hourly Wage - Pct25':[hourly['Pct25']]
            ,'Hourly Wage - Median':[hourly['Median']]
            ,'Hourly Wage - Pct75':[hourly['Pct75']]
            ,'Hourly Wage - Pct90':[hourly['Pct90']]
            ,'WageYear':[response['OccupationDetail'][0]['Wages']['WageYear']]
            ,'BrightOutlook':[response['OccupationDetail'][0]['BrightOutlook']]
            ,'EducationLevel_1':[EducationLevel_1]
            ,'EducationLevel_2':[EducationLevel_2]
            ,'EducationLevel_3':[EducationLevel_3]
            ,'EducationLevel_4':[EducationLevel_4]
            ,'EducationLevel_5':[EducationLevel_5]
            ,'EducationLevel_6':[EducationLevel_6]
            ,'EducationLevel_7':[EducationLevel_7]
            ,'EducationCode':[response['OccupationDetail'][0]['EducationTraining']['EducationCode']]
            ,'EducationTitle':[response['OccupationDetail'][0]['EducationTraining']['EducationTitle']]
            ,'ExperienceCode':[response['OccupationDetail'][0]['EducationTraining']['ExperienceCode']]
            ,'ExperienceTitle':[response['OccupationDetail'][0]['EducationTraining']['ExperienceTitle']]
            ,'TrainingCode':[response['OccupationDetail'][0]['EducationTraining']['TrainingCode']]
            ,'TrainingTitle':[response['OccupationDetail'][0]['EducationTraining']['TrainingTitle']]
            ,'EducationOccupationTitle':[response['OccupationDetail'][0]['EducationTraining']['OccupationTitle']]
            }, index=[o_net_df['Code'][i]])
    occupation_df = occupation_df.append(df)
    i += 1

occupation_df.head()

,Occupation,Career Pathway,OnetTitle,OnetDescription,Annual Wage - Pct10,Annual Wage - Pct25,Annual Wage - Median,Annual Wage - Pct75,Annual Wage - Pct90,Hourly Wage - Pct10,...,EducationLevel_5,EducationLevel_6,EducationLevel_7,EducationCode,EducationTitle,ExperienceCode,ExperienceTitle,TrainingCode,TrainingTitle,OccupationTitle
15-2051.01,Business Intelligence Analysts,Information Support and Services,Business Intelligence Analysts,Produce financial and market intelligence by q...,44810,54300,64250,98980,138330,21.55,...,37.5,36,14.5,3,Bachelor's degree,3,No work experience,6,No on-the-job training,Data Scientists and Mathematical Science Occup...
15-1299.00,"Computer Occupations, All Other",Information Support and Services,"Computer Occupations, All Other",All computer occupations not listed separately.,31020,45760,71250,98880,123080,14.91,...,43.6,20,2.2,3,Bachelor's degree,3,No work experience,6,No on-the-job training,"Computer Occupations, All Other"
15-1299.08,Computer Systems Engineers/Architects,Information Support and Services,Computer Systems Engineers/Architects,Design and develop solutions to complex applic...,31020,45760,71250,98880,123080,14.91,...,43.6,20,2.2,3,Bachelor's degree,3,No work experience,6,No on-the-job training,"Computer Occupations, All Other"
15-1232.00,Computer User Support Specialists,Information Support and Services,Computer User Support Specialists,Provide technical assistance to computer users...,30040,36570,46970,61660,79350,14.44,...,37,10.7,1,6,"Some college, no degree",3,No work experience,6,No on-the-job training,Computer User Support Specialists
15-1243.01,Data Warehousing Specialists,Information Support and Services,Data Warehousing Specialists,"Design, model, or implement corporate data war...",45430,63700,90230,116170,134290,21.84,...,46.9,25.2,2.7,3,Bachelor's degree,3,No work experience,6,No on-the-job training,Database Administrators and Architects


## Step 3: Print to csv

In [8]:
# Write dataframe to CSV, without the Pandas index
occupation_df.to_csv("Clean_Data/CareerOneStop.csv", index=False, header=True)